# Setup Genie Space, service principal and save secrets


1. Manually create a genie space with the four revenue tables
2. Create service principal permissions to this space
3. Copy the current databricks host
4. Store the secrets

In [0]:
import requests
import json
import os

# Set your Databricks workspace URL and token
DATABRICKS_URL = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
SCOPE = "botops"


def save_secret(scope, key, value):
    response = requests.post(
        f"{DATABRICKS_URL}/api/2.0/secrets/put",
        headers={"Authorization": f"Bearer {DATABRICKS_TOKEN}"},
        json={"scope": scope, "key": key, "string_value": value}
    )
    response.raise_for_status()

In [0]:
# Create a secret scope
response = requests.post(
        f"{DATABRICKS_URL}/api/2.0/secrets/scopes/create",
        headers={"Authorization": f"Bearer {DATABRICKS_TOKEN}"},
        json={"scope": SCOPE}
)
response.raise_for_status()

In [0]:
#  Define values for these
genie_space_id = 
databricks_host = 
# service_principal_secret = 
# service_principal_client_id = 

In [0]:
save_secret(SCOPE, "botopsgenie_geniespace_id", genie_space_id)
save_secret(SCOPE, "botopsgenie_databricks_host", databricks_host)
save_secret(SCOPE, "botopsgenie_service_principal_id", service_principal_id)
save_secret(SCOPE, "botopsgenie_service_secret", service_principal_secret)

Give user group access to read scope

In [0]:
import json# Define the secrets scope and permissions

scope_name = SCOPE
permissions = {
    "access_control_list": [
        # {
        #     "group_name": "admins",
        #     "permission_level": "MANAGE"
        # },
        {
            "group_name": "dataops-workshop",
            "permission_level": "READ"
        }
    ]
}

# Set the Databricks host and token
# Define the API endpoint
api_url = f"{DATABRICKS_URL}/api/2.0/secrets/acls/put"

# Set the headers
headers = {
    "Authorization": f"Bearer {DATABRICKS_TOKEN}",
    "Content-Type": "application/json"
}

# Set the permissions for each group
for acl in permissions["access_control_list"]:
    payload = {
        "scope": scope_name,
        "principal": acl["group_name"],
        "permission": acl["permission_level"]
    }
    response = requests.post(api_url, headers=headers, data=json.dumps(payload))
    if response.status_code != 200:
        raise Exception(f"Error setting permission: {response.text}")

display("Secrets scope permissions set successfully.")